In [ ]:
# ============================================================================
# PARÁMETROS - Esta celda debe tener tag "parameters"
# ============================================================================

# Estos son los valores DEFAULT (papermill los va a sobreescribir)
ventana_nombre <- "6m"
ventana_meses <- 6

# También podés agregar otros parámetros si querés
experimento_nombre <- "stacking_ventanas"

In [ ]:
# Celda siguiente (ejemplo)
cat("=== Ejecutando ventana:", ventana_nombre, "===\n")
cat("Cantidad de meses:", ventana_meses, "\n")

# Tu código existente, modificado para usar ventana_meses
for (mes_objetivo in 202102:202106) {
  
  # Calcular meses de entrenamiento según ventana
  if (ventana_meses == 999) {  # Código especial para "full"
    meses_train <- 201901:(mes_objetivo - 1)
  } else {
    mes_inicio <- max(201901, mes_objetivo - ventana_meses)
    meses_train <- mes_inicio:(mes_objetivo - 1)
  }
  
  cat("Prediciendo", mes_objetivo, "con datos de", 
      min(meses_train), "a", max(meses_train), "\n")
  
  # Tu código de entrenamiento aquí...
  # modelo <- lgb.train(data = dataset[foto_mes %in% meses_train], ...)
  # preds <- predict(modelo, ...)
  # dataset[foto_mes == mes_objetivo, paste0("pred_", ventana_nombre) := preds]
}

In [ ]:
# Última celda - guardar resultados
fwrite(
  dataset[foto_mes %in% 202102:202106, 
          .(foto_mes, numero_de_cliente, 
            get(paste0("pred_", ventana_nombre)))],
  file = paste0("./resultados/predicciones_", ventana_nombre, ".csv")
)

cat("✓ Predicciones de ventana", ventana_nombre, "guardadas\n")

In [ ]:
library(data.table)

# Crear carpeta de resultados si no existe
if (!dir.exists("./resultados")) dir.create("./resultados")
if (!dir.exists("./resultados/notebooks_ejecutados")) {
  dir.create("./resultados/notebooks_ejecutados")
}

# Orquestador

In [ ]:
library(data.table)

# Crear carpeta de resultados si no existe
if (!dir.exists("./resultados")) dir.create("./resultados")
if (!dir.exists("./resultados/notebooks_ejecutados")) {
  dir.create("./resultados/notebooks_ejecutados")
}

In [ ]:
# Definir las ventanas que queremos ejecutar
ventanas <- data.table(
  nombre = c("6m", "12m", "full"),
  meses = c(6, 12, 999)  # 999 es código para "usar todo el histórico"
)

print(ventanas)

In [ ]:
# Ejecutar notebook base para cada ventana
for (i in 1:nrow(ventanas)) {
  
  v_nombre <- ventanas[i, nombre]
  v_meses <- ventanas[i, meses]
  
  cat("\n========================================\n")
  cat("EJECUTANDO VENTANA:", v_nombre, "\n")
  cat("========================================\n\n")
  
  # Construir comando papermill
  input_notebook <- "workflow_ventanas.ipynb"
  output_notebook <- paste0("./resultados/notebooks_ejecutados/output_", v_nombre, ".ipynb")
  
  comando <- sprintf(
    "papermill %s %s -p ventana_nombre %s -p ventana_meses %d",
    input_notebook,
    output_notebook,
    v_nombre,
    v_meses
  )
  
  cat("Comando:", comando, "\n\n")
  
  # Ejecutar
  resultado <- system(comando, intern = FALSE)
  
  if (resultado == 0) {
    cat("✓ Ventana", v_nombre, "completada exitosamente\n")
  } else {
    cat("✗ ERROR en ventana", v_nombre, "\n")
    stop("Ejecución detenida por error")
  }
}

cat("\n=== TODAS LAS VENTANAS COMPLETADAS ===\n")

In [ ]:
# Cargar todas las predicciones generadas
cat("\n=== Consolidando predicciones ===\n")

archivos_predicciones <- list.files(
  "./resultados", 
  pattern = "predicciones_.*\\.csv",
  full.names = TRUE
)

cat("Archivos encontrados:\n")
print(archivos_predicciones)

# Cargar primer archivo
dataset_consolidado <- fread(archivos_predicciones[1])
setnames(dataset_consolidado, old = "V3", new = gsub(".*_(.*)\\.csv", "pred_\\1", archivos_predicciones[1]))

# Hacer merge con los demás
for (i in 2:length(archivos_predicciones)) {
  
  pred_temp <- fread(archivos_predicciones[i])
  v_nombre <- gsub(".*predicciones_(.*)\\.csv", "\\1", archivos_predicciones[i])
  setnames(pred_temp, old = "V3", new = paste0("pred_", v_nombre))
  
  dataset_consolidado <- merge(
    dataset_consolidado,
    pred_temp,
    by = c("foto_mes", "numero_de_cliente")
  )
}

cat("\nDataset consolidado:\n")
cat("  Filas:", nrow(dataset_consolidado), "\n")
cat("  Columnas:", ncol(dataset_consolidado), "\n")
print(head(dataset_consolidado, 3))

In [ ]:
# Cargar dataset original para agregar features raw
dataset_original <- fread("tu_dataset.csv")  # Ajustar path

# Merge con features raw
dataset_meta <- merge(
  dataset_consolidado,
  dataset_original[foto_mes %in% 202102:202106],
  by = c("foto_mes", "numero_de_cliente")
)

# Guardar dataset final para meta-modelo
fwrite(dataset_meta, "./resultados/dataset_meta_modelo.csv")

cat("\n✓ Dataset para meta-modelo guardado\n")
cat("  Archivo: ./resultados/dataset_meta_modelo.csv\n")
cat("  Predicciones disponibles:", 
    paste(grep("^pred_", colnames(dataset_meta), value = TRUE), collapse = ", "), "\n")

# Debug y más

In [ ]:
# En el notebook base, ejecutar esto en una celda:
system("jupyter nbconvert --to notebook --execute workflow_ventanas.ipynb")

In [ ]:
# Con progreso más detallado
comando <- sprintf(
  "papermill %s %s -p ventana_nombre %s -p ventana_meses %d --log-output",
  input_notebook, output_notebook, v_nombre, v_meses
)

# Con timeout (si una ventana tarda mucho)
comando <- sprintf(
  "papermill %s %s -p ventana_nombre %s -p ventana_meses %d --execution-timeout 3600",
  input_notebook, output_notebook, v_nombre, v_meses
)  # 3600 segundos = 1 hora
```

### Estructura de archivos final:
```
proyecto/
├── workflow_ventanas.ipynb          # Tu notebook base (parametrizado)
├── run_stacking_ventanas.ipynb      # Orquestador
├── tu_dataset.csv                   # Dataset original
└── resultados/
    ├── predicciones_6m.csv
    ├── predicciones_12m.csv
    ├── predicciones_full.csv
    ├── dataset_meta_modelo.csv      # ← Este es el que necesitás
    └── notebooks_ejecutados/
        ├── output_6m.ipynb
        ├── output_12m.ipynb
        └── output_full.ipynb